In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error


class TimestampsDataset(torch.utils.data.Dataset):
    def __init__(self, data: Path, lags: int) -> None:
        super(TimestampsDataset, self).__init__()
        self.data = pd.read_csv(data, index_col=0, parse_dates=["Date"])
        self.lags = lags
        
    def __len__(self) -> int:
        return len(self.data) - self.lags
    
    def __getitem__(self, index: int) -> torch.Tensor:
        return self.data.iloc[index:index+self.lags].to_numpy(dtype=np.float32)


class LSTMForecasterSubhead(nn.Module):
    def __init__(self, hidden_size: int = 256, num_layers: int = 2, dropout: float = 0.2) -> None:
        super(LSTMForecasterSubhead, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=6,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        
        self.head = nn.Linear(
            hidden_size,
            240
        )
        
    def forward(self, x) -> torch.Tensor:
        out, _ = self.lstm(x)
        
        out = out[:, -1, :]  # take last time step
        out = self.head(out)
        
        return out


class LSTMForecaster(nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        
        self.series_1_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        self.series_2_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        self.series_3_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        self.series_4_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        self.series_5_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
        self.series_6_head = LSTMForecasterSubhead(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        predicted_series_1: torch.Tensor = self.series_1_head(x)
        predicted_series_2: torch.Tensor = self.series_2_head(x)
        predicted_series_3: torch.Tensor = self.series_3_head(x)
        predicted_series_4: torch.Tensor = self.series_4_head(x)
        predicted_series_5: torch.Tensor = self.series_5_head(x)
        predicted_series_6: torch.Tensor = self.series_6_head(x)
        
        y = torch.stack(
            [
                predicted_series_1,
                predicted_series_2,
                predicted_series_3,
                predicted_series_4,
                predicted_series_5,
                predicted_series_6
            ],
            1
        )
        
        return y.permute(0, 2, 1)
    
    
def val_step(
    logits: torch.Tensor,
    labels: torch.Tensor,
    criterion: torch.nn.MSELoss
) -> torch.Tensor:
    loss = criterion(logits, labels)
    return loss


def train_step(
    logits: torch.Tensor,
    labels: torch.Tensor,
    criterion: torch.nn.MSELoss,
    optimizer: torch.optim.Optimizer
) -> torch.Tensor:
    optimizer.zero_grad()
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    return loss


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    epochs: int = 20
) -> None:
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        
        losses = []
        r2s = []
        mses = []
        rmses = []
        
        model.train()
        
        train_tqdm = tqdm(train_dataloader, total=len(train_dataloader))
        
        for full_batch in train_tqdm:
            
            inputs = full_batch[:, :-240, :]
            labels = full_batch[:, -240:, :]
            
            logits: torch.Tensor = model(inputs)
            
            loss = train_step(logits, labels, criterion, optimizer)
            
            eval_labels = labels.detach().numpy().reshape(train_dataloader.batch_size, -1)
            eval_logits = logits.detach().numpy().reshape(train_dataloader.batch_size, -1)
            
            r2 = r2_score(eval_logits, eval_labels)
            mse = mean_absolute_error(eval_logits, eval_labels)
            rmse = root_mean_squared_error(eval_logits, eval_labels)
            
            r2s.append(r2)
            mses.append(mse)
            rmses.append(rmse)
            losses.append(loss.item())
            
            train_tqdm.set_description(
                f"Train Epoch {epoch}, Loss - {np.mean(losses):0.4f}, R^2 - {np.mean(r2s):0.4f}, MSE - {np.mean(mses):0.4f}, RMSE - {np.mean(rmses):0.4f}"
            )
            
        r2s.clear()
        mses.clear()
        rmses.clear()
        losses.clear()
        
        with torch.no_grad():
            
            model.eval()
            
            test_tqdm = tqdm(test_dataloader, total=len(test_dataloader))
            
            for full_batch in test_tqdm:
                
                inputs = full_batch[:, :-1, :]
                labels = full_batch[:, -1, :]
                
                logits = model(inputs)

                loss = val_step(logits, labels, criterion)
            
                eval_labels = labels.detach().numpy().reshape(test_dataloader.batch_size, -1)
                eval_logits = logits.detach().numpy().reshape(test_dataloader.batch_size, -1)
                
                r2 = r2_score(eval_logits, eval_labels)
                mse = mean_absolute_error(eval_logits, eval_labels)
                rmse = root_mean_squared_error(eval_logits, eval_labels)
                
                r2s.append(r2)
                mses.append(mse)
                rmses.append(rmse)
                losses.append(loss.item())
                
                test_tqdm.set_description(
                    f"Test, Loss - {np.mean(losses):0.4f}, R^2 - {np.mean(r2s):0.4f}, MSE - {np.mean(mses):0.4f}, RMSE - {np.mean(rmses):0.4f}"
                )
                
        print()

In [ ]:
INPUT_SEQUENCE = 32
OUTPUT_SEQUENCE = 240

model = LSTMForecaster()
train_dataset = TimestampsDataset(data=Path("train.csv"), lags=INPUT_SEQUENCE + OUTPUT_SEQUENCE)
test_dataset = TimestampsDataset(data=Path("test.csv"), lags=INPUT_SEQUENCE + OUTPUT_SEQUENCE)

train(
    model, 
    train_dataloader=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        drop_last=True
    ),
    test_dataloader=torch.utils.data.DataLoader(
        test_dataset,
        batch_size=4,
        shuffle=True,
        drop_last=True
    )
)

Train Epoch 0, Loss - 0.9346, R^2 - -113.5709, MSE - 0.6508, RMSE - 0.8959:   2%|▏         | 895/58045 [00:46<49:05, 19.40it/s]  


KeyboardInterrupt: 